<a href="https://colab.research.google.com/github/efrat-dev/insider-threat-detector/blob/main/EDA_13_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import random
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

df = pd.read_csv("insider_threat_dataset.csv")

# EDA Pipeline
print("\n" + "="*50)
print("EXPLORATORY DATA ANALYSIS")
print("="*50)

# 1. BASIC DATASET INFORMATION
print("\n1. BASIC DATASET INFORMATION")
print("-" * 30)

# Check: Basic dataset structure and missing values
print("Checking dataset structure and missing values...")
print(f"Dataset shape: {df.shape}")
print(f"Missing values per column:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

# 2. EMPLOYEE BACKGROUND CHARACTERISTICS
print("\n2. EMPLOYEE BACKGROUND CHARACTERISTICS")
print("-" * 40)

# Check: Medical history distribution
print("Checking medical history distribution...")
medical_counts = df.groupby('employee_id')['has_medical_history'].first().value_counts()
medical_ratio = medical_counts[1] / (medical_counts[0] + medical_counts[1]) * 100
print(f"Medical history ratio: {medical_ratio:.2f}% have medical history, {100-medical_ratio:.2f}% don't have medical history")

# Check: Criminal record distribution
print("Checking criminal record distribution...")
criminal_counts = df.groupby('employee_id')['has_criminal_record'].first().value_counts()
criminal_ratio = criminal_counts[1] / (criminal_counts[0] + criminal_counts[1]) * 100
print(f"Criminal record ratio: {criminal_ratio:.2f}% have criminal record, {100-criminal_ratio:.2f}% don't have criminal record")

# Check: Foreign citizenship distribution
print("Checking foreign citizenship distribution...")
foreign_counts = df.groupby('employee_id')['has_foreign_citizenship'].first().value_counts()
foreign_ratio = foreign_counts[1] / (foreign_counts[0] + foreign_counts[1]) * 100
print(f"Foreign citizenship ratio: {foreign_ratio:.2f}% have foreign citizenship, {100-foreign_ratio:.2f}% don't have foreign citizenship")

# 3. ACTIVITY PATTERNS ANALYSIS
print("\n3. ACTIVITY PATTERNS ANALYSIS")
print("-" * 35)

# Check: Activity vs no activity days ratio per employee
print("Checking ratio of activity days vs no activity days per employee...")
def has_any_activity(row):
    """Check if employee had any activity on a given day"""
    return (row['num_print_commands'] > 0 or
            row['num_burn_requests'] > 0 or
            row['is_abroad'] > 0 or
            row['num_entries'] > 0)

df['has_activity'] = df.apply(has_any_activity, axis=1)

activity_by_employee = df.groupby('employee_id')['has_activity'].agg(['sum', 'count'])
activity_by_employee['activity_ratio'] = activity_by_employee['sum'] / activity_by_employee['count'] * 100
activity_by_employee['no_activity_ratio'] = 100 - activity_by_employee['activity_ratio']

avg_activity_ratio = activity_by_employee['activity_ratio'].mean()
avg_no_activity_ratio = activity_by_employee['no_activity_ratio'].mean()

print(f"Average ratio per employee: {avg_activity_ratio:.2f}% activity days, {avg_no_activity_ratio:.2f}% no activity days")

# Check: Percentage of employees with no activity records
print("Checking percentage of employees with some no-activity records...")
employees_with_no_activity = (activity_by_employee['no_activity_ratio'] > 0).sum()
total_employees = len(activity_by_employee)
pct_employees_with_no_activity = employees_with_no_activity / total_employees * 100
print(f"{pct_employees_with_no_activity:.2f}% of employees have some days with no activity")

# Check: Missing records for specific dates
print("Checking for missing records on specific dates...")
expected_records_per_date = df['employee_id'].nunique()
actual_records_per_date = df.groupby('date').size()
missing_records = expected_records_per_date - actual_records_per_date
dates_with_missing = (missing_records > 0).sum()
print(f"Number of dates with missing employee records: {dates_with_missing}")
if dates_with_missing > 0:
    print(f"Maximum missing records on any date: {missing_records.max()}")

# 4. MALICIOUS VS NON-MALICIOUS PATTERNS
print("\n4. MALICIOUS VS NON-MALICIOUS PATTERNS")
print("-" * 40)

# Check: Average malicious days for malicious vs non-malicious employees
print("Checking average malicious activity days...")
malicious_days_by_employee = df.groupby('employee_id')['is_malicious'].sum()
employee_types = df.groupby('employee_id')['is_malicious'].first()

malicious_employees_avg_days = malicious_days_by_employee[employee_types == 1].mean()
non_malicious_employees_avg_days = malicious_days_by_employee[employee_types == 0].mean()

print(f"Average malicious days for malicious employees: {malicious_employees_avg_days:.2f}")
print(f"Average malicious days for non-malicious employees: {non_malicious_employees_avg_days:.2f}")

# 5. BUILDING ACCESS VALIDATION
print("\n5. BUILDING ACCESS VALIDATION")
print("-" * 35)

# Check: Entry-exit balance
print("Checking entry-exit balance...")
access_records = df[df['num_entries'].notna()].copy()
balanced_entries = (access_records['num_entries'] == access_records['num_exits']).sum()
total_access_records = len(access_records)
balanced_ratio = balanced_entries / total_access_records * 100
unbalanced_ratio = 100 - balanced_ratio

print(f"Entry-exit balance: {balanced_ratio:.2f}% balanced, {unbalanced_ratio:.2f}% unbalanced")

# Check: Unbalanced entry-exit patterns
print("Analyzing unbalanced entry-exit patterns...")
unbalanced_records = access_records[access_records['num_entries'] != access_records['num_exits']]
total_unbalanced = len(unbalanced_records)

if total_unbalanced > 0:
    more_entries = (unbalanced_records['num_entries'] > unbalanced_records['num_exits']).sum()
    more_exits = (unbalanced_records['num_exits'] > unbalanced_records['num_entries']).sum()

    more_entries_pct = more_entries / total_unbalanced * 100
    more_exits_pct = more_exits / total_unbalanced * 100

    print(f"Unbalanced records breakdown:")
    print(f"  - {unbalanced_ratio:.2f}% of all records are unbalanced")
    print(f"  - Of unbalanced records: {more_entries_pct:.2f}% have more entries, {more_exits_pct:.2f}% have more exits")

# 6. PRINTING ACTIVITY VALIDATION
print("\n6. PRINTING ACTIVITY VALIDATION")
print("-" * 35)

# Check: Off-hours pages > total pages
print("Checking if off-hours printed pages exceed total pages...")
invalid_off_hours_pages = (df['num_printed_pages_off_hours'] > df['total_printed_pages']).sum()
total_records = len(df)
invalid_off_hours_pages_pct = invalid_off_hours_pages / total_records * 100
print(f"{invalid_off_hours_pages_pct:.2f}% of records have off-hours pages exceeding total pages")

# Check: Off-hours commands > total commands
print("Checking if off-hours commands exceed total commands...")
invalid_off_hours_commands = (df['num_print_commands_off_hours'] > df['num_print_commands']).sum()
invalid_off_hours_commands_pct = invalid_off_hours_commands / total_records * 100
print(f"{invalid_off_hours_commands_pct:.2f}% of records have off-hours commands exceeding total commands")

# Check: Commands > pages
print("Checking if print commands exceed printed pages...")
commands_exceed_pages = (df['num_print_commands'] > df['total_printed_pages']).sum()
commands_exceed_pages_pct = commands_exceed_pages / total_records * 100
print(f"{commands_exceed_pages_pct:.2f}% of records have commands exceeding pages")

# Check: Color pages > total pages
print("Checking if color pages exceed total pages...")
color_exceed_total = (df['num_color_prints'] > df['total_printed_pages']).sum()
color_exceed_total_pct = color_exceed_total / total_records * 100
print(f"{color_exceed_total_pct:.2f}% of records have color pages exceeding total pages")

# 7. PRINTING RATIOS ANALYSIS
print("\n7. PRINTING RATIOS ANALYSIS")
print("-" * 30)

# Check: Off-hours to regular hours ratio
print("Checking average ratio of off-hours to regular hours printing...")
records_with_off_hours = df[(df['num_printed_pages_off_hours'] > 0) & (df['total_printed_pages'] > 0)].copy()
if len(records_with_off_hours) > 0:
    records_with_off_hours['regular_hours_pages'] = (records_with_off_hours['total_printed_pages'] -
                                                   records_with_off_hours['num_printed_pages_off_hours'])
    records_with_off_hours_positive = records_with_off_hours[records_with_off_hours['regular_hours_pages'] > 0]

    if len(records_with_off_hours_positive) > 0:
        avg_off_hours_ratio = (records_with_off_hours_positive['num_printed_pages_off_hours'] /
                             records_with_off_hours_positive['regular_hours_pages']).mean()
        print(f"Average ratio of off-hours to regular hours printing: {avg_off_hours_ratio:.2f}")
    else:
        print("No records with both off-hours and regular hours printing")
else:
    print("No records with off-hours printing")

# Check: Off-hours commands ratio
print("Checking average ratio of off-hours to total commands...")
records_with_off_hours_cmds = df[(df['num_print_commands_off_hours'] > 0) & (df['num_print_commands'] > 0)].copy()
if len(records_with_off_hours_cmds) > 0:
    avg_off_hours_cmd_ratio = (records_with_off_hours_cmds['num_print_commands_off_hours'] /
                              records_with_off_hours_cmds['num_print_commands']).mean()
    print(f"Average ratio of off-hours to total commands: {avg_off_hours_cmd_ratio:.2f}")
else:
    print("No records with off-hours commands")

# Check: Color to B&W ratio
print("Checking average ratio of color to B&W printing...")
records_with_color = df[(df['num_color_prints'] > 0) & (df['num_bw_prints'] > 0)].copy()
if len(records_with_color) > 0:
    avg_color_bw_ratio = (records_with_color['num_color_prints'] /
                         records_with_color['num_bw_prints']).mean()
    print(f"Average ratio of color to B&W printing: {avg_color_bw_ratio:.2f}")
else:
    print("No records with both color and B&W printing")

# 8. PRINTING CONSISTENCY CHECKS
print("\n8. PRINTING CONSISTENCY CHECKS")
print("-" * 35)

# Check: Color + B&W = Total pages consistency
print("Checking if color + B&W pages equal total pages...")
print_records = df[df['total_printed_pages'] > 0].copy()
if len(print_records) > 0:
    color_bw_sum = print_records['num_color_prints'] + print_records['num_bw_prints']
    inconsistent_totals = (color_bw_sum != print_records['total_printed_pages']).sum()
    inconsistent_totals_pct = inconsistent_totals / len(print_records) * 100
    print(f"{inconsistent_totals_pct:.2f}% of printing records have inconsistent color+B&W totals")
else:
    print("No printing records found")

# Check: Color ratio consistency
print("Checking if color ratio matches actual color percentage...")
print_records_with_ratio = df[(df['total_printed_pages'] > 0) & (df['ratio_color_prints'] > 0)].copy()
if len(print_records_with_ratio) > 0:
    calculated_ratio = print_records_with_ratio['num_color_prints'] / print_records_with_ratio['total_printed_pages']
    ratio_diff = abs(calculated_ratio - print_records_with_ratio['ratio_color_prints'])
    # Allow small floating point differences
    inconsistent_ratios = (ratio_diff > 0.01).sum()
    inconsistent_ratios_pct = inconsistent_ratios / len(print_records_with_ratio) * 100
    print(f"{inconsistent_ratios_pct:.2f}% of records have inconsistent color ratios")
else:
    print("No records with color printing and ratios found")

# Check: Commands vs pages relationship
print("Checking relationship between print commands and pages...")
print_records_all = df[df['num_print_commands'] > 0].copy()
if len(print_records_all) > 0:
    commands_not_equal_pages = (print_records_all['num_print_commands'] != print_records_all['total_printed_pages']).sum()
    commands_not_equal_pages_pct = commands_not_equal_pages / len(print_records_all) * 100
    print(f"{commands_not_equal_pages_pct:.2f}% of records have different number of commands vs pages")

    # This is actually expected - commands and pages should often be different
    # Let's show some statistics
    avg_pages_per_command = (print_records_all['total_printed_pages'] / print_records_all['num_print_commands']).mean()
    print(f"Average pages per command: {avg_pages_per_command:.2f}")
else:
    print("No records with print commands found")

# 9. SUMMARY STATISTICS
print("\n9. SUMMARY STATISTICS")
print("-" * 25)

print("Dataset validation summary:")
print(f"- Total records: {len(df):,}")
print(f"- Unique employees: {df['employee_id'].nunique():,}")
print(f"- Date range: {df['date'].nunique()} days")
print(f"- Malicious employee ratio: {df.groupby('employee_id')['is_malicious'].first().mean()*100:.2f}%")

# Activity summary
activity_summary = df.groupby('employee_id')['has_activity'].agg(['sum', 'count', 'mean'])
print(f"- Average activity days per employee: {activity_summary['sum'].mean():.1f}")
print(f"- Average activity rate per employee: {activity_summary['mean'].mean()*100:.1f}%")

# Print activity summary
print_summary = df[df['total_printed_pages'] > 0]
if len(print_summary) > 0:
    print(f"- Records with printing activity: {len(print_summary):,} ({len(print_summary)/len(df)*100:.1f}%)")
    print(f"- Average pages per printing day: {print_summary['total_printed_pages'].mean():.1f}")

# Access summary
access_summary = df[df['num_entries'] > 0]
if len(access_summary) > 0:
    print(f"- Records with building access: {len(access_summary):,} ({len(access_summary)/len(df)*100:.1f}%)")
    print(f"- Average presence time: {access_summary['total_presence_minutes'].mean():.0f} minutes")

# Check contractor ratio
contractor_counts = df['is_contractor'].value_counts()
print(f"\nContractor vs Regular Employee Ratio:")
print(f"Regular Employees (0): {contractor_counts[0]:,} ({contractor_counts[0]/len(df)*100:.1f}%)")
print(f"Contractors (1): {contractor_counts[1]:,} ({contractor_counts[1]/len(df)*100:.1f}%)")

# Check classification levels distribution
classification_counts = df['employee_classification'].value_counts().sort_index()
print(f"\nClassification Levels Distribution:")
for level in sorted(classification_counts.index):
    count = classification_counts[level]
    print(f"Classification {level}: {count:,} ({count/len(df)*100:.1f}%)")

# Check inconsistent avg vs max request classification
inconsistent_rows = df[(df['avg_request_classification'] > df['max_request_classification']) &
                      (df['avg_request_classification'] > 0)]
print(f"\nRows where avg_request_classification > max_request_classification: {len(inconsistent_rows)}")
if len(inconsistent_rows) > 0:
    print("Sample of inconsistent rows:")
    print(inconsistent_rows[['employee_id', 'date', 'avg_request_classification', 'max_request_classification']].head())

print("\n" + "="*50)
print("EDA COMPLETE")
print("="*50)


EXPLORATORY DATA ANALYSIS

1. BASIC DATASET INFORMATION
------------------------------
Checking dataset structure and missing values...
Dataset shape: (7657, 43)
Missing values per column:
has_criminal_record                1
has_medical_history                1
is_malicious                       1
print_location_campus           5164
num_print_commands                 1
total_printed_pages                1
num_print_commands_off_hours       1
num_printed_pages_off_hours        1
num_color_prints                   1
num_bw_prints                      1
ratio_color_prints                 1
burn_location_campus            7227
num_burn_requests                  1
max_request_classification         1
avg_request_classification         1
num_burn_requests_off_hours        1
total_burn_volume_mb               1
total_files_burned                 1
is_abroad                          1
trip_day_number                 7517
country_name                    7517
is_hostile_country_trip          